In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.widgets import LassoSelector
from matplotlib.path import Path
from ipywidgets import FileUpload, Button, Output
from IPython.display import display
import io

# Enable interactive matplotlib
%matplotlib widget

# Create a file upload widget
upload = FileUpload(accept='.jpg,.png', multiple=False)
display(upload)

# Create a button to trigger processing
button = Button(description="Process Image")
output = Output()

def process_image(upload_widget):
    """Reads an uploaded image file and converts it to a NumPy array."""
    if not upload_widget.value:
        print("Error: No image uploaded.")
        return None

    # Debug: Print the type and structure of upload.value
    print("Debug: upload.value type:", type(upload_widget.value))
    print("Debug: upload.value content:", upload_widget.value)

    # Extract the uploaded file correctly
    file_data = list(upload_widget.value.values())[0]  # Get the first file
    file_bytes = file_data['content']  # Extract byte content

    # Convert byte content into a NumPy array
    image_array = np.frombuffer(file_bytes, np.uint8)

    # Decode image using OpenCV
    image = cv2.imdecode(image_array, cv2.IMREAD_COLOR)

    if image is None:
        print("Error: Failed to decode image.")
        return None

    return image

def on_button_click(b):
    """Handles button click to process the uploaded image."""
    with output:
        output.clear_output()  # Clear previous outputs

        # Process the uploaded image
        image = process_image(upload)

        if image is None:
            print("Please upload a valid image file.")
            return

        # Convert the image from BGR to RGB (for matplotlib display)
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        # Create a matplotlib figure
        fig, ax = plt.subplots()
        ax.imshow(image_rgb)
        ax.set_title("Draw a freehand region to measure")

        # Initialize variables to store the selected region
        selected_pixels = []

        def onselect(verts):
            """Handles Lasso selection and extracts selected pixel coordinates."""
            nonlocal selected_pixels
            path = Path(verts)
            x, y = np.meshgrid(np.arange(image.shape[1]), np.arange(image.shape[0]))
            x, y = x.flatten(), y.flatten()
            points = np.vstack((x, y)).T
            mask = path.contains_points(points)
            selected_pixels = points[mask]
            plt.close()

        # Create a LassoSelector widget
        lasso = LassoSelector(ax, onselect)

        # Add a button to confirm the selection
        confirm_button = Button(description="Confirm Selection")
        confirm_output = Output()

        def on_confirm_click(b):
            """Handles confirm button click and processes the selected region."""
            with confirm_output:
                confirm_output.clear_output()  # Clear previous outputs

                if len(selected_pixels) == 0:
                    print("No region selected.")
                    return

                # Create a mask for the selected region
                mask = np.zeros_like(image[:, :, 0], dtype=np.uint8)
                for x, y in selected_pixels:
                    mask[int(y), int(x)] = 255

                # Apply the mask to the image
                masked_image = cv2.bitwise_and(image, image, mask=mask)

                # Find contours of the selected region
                contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

                # Draw the bounding box around the selected region
                for contour in contours:
                    x, y, w, h = cv2.boundingRect(contour)
                    cv2.rectangle(masked_image, (x, y), (x + w, y + h), (0, 255, 0), 2)

                    # Print the size of the bounding box (in pixels)
                    print(f"Selected region size: {w}px (width) x {h}px (height)")

                # Display the masked image with the bounding box
                plt.imshow(cv2.cvtColor(masked_image, cv2.COLOR_BGR2RGB))
                plt.axis('off')
                plt.show()

                # Optional: Convert pixel size to real-world units (e.g., millimeters)
                dpi = 300  # Replace with the DPI of your image
                mm_per_inch = 25.4
                width_mm = (w / dpi) * mm_per_inch
                height_mm = (h / dpi) * mm_per_inch
                print(f"Selected region size: {width_mm:.2f}mm (width) x {height_mm:.2f}mm (height)")

        confirm_button.on_click(on_confirm_click)

        # Display the confirm button and output
        display(confirm_button, confirm_output)

        # Show the plot
        plt.show()

button.on_click(on_button_click)

# Display the button and output
display(button, output)
